In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Load data
prices_df = pd.read_csv('data\Russell3000_prices_clean.csv')
prices_df['Date'] = pd.to_datetime(prices_df['Date'], dayfirst=True)
prices_df.sort_values(by='Date', inplace=True)

# Select stocks to use; exclude the first column (date)
selected_stocks = prices_df.columns[1:20]

# Calculate returns
returns_df = prices_df[selected_stocks].pct_change()
returns_df = returns_df.dropna().reset_index(drop=True)
returns_df.insert(0, 'Date', prices_df['Date'][:-1])

# Convert to numpy array (for easier calculations)
returns_matrix = returns_df.iloc[:, 1:].to_numpy()

In [45]:
test_start_date = pd.to_datetime('2020-01-01')
test_end_date = pd.to_datetime('2021-07-15')

messages = {"test_start": [], "test_end": []}

# Check if test_start_date exists in the DataFrame, if not, take the next date that exists
if test_start_date not in returns_df['Date'].values:
    test_start_date = returns_df['Date'][returns_df['Date'] > test_start_date].bfill().iloc[0]
    messages["test_start"] = messages["test_start"] or ['Start date not found, using closest next date instead.']

# Check if test_end_date exists in the DataFrame, if not, take the previous date that exists
if test_end_date not in returns_df['Date'].values:
    test_end_date = returns_df['Date'][returns_df['Date'] < test_end_date].ffill().iloc[-1]
    messages["test_end"] = messages["test_end"] or ['End date not found, using closest date instead.']

test_start = returns_df.loc[returns_df['Date'] == test_start_date].index[0]
test_end = returns_df.loc[returns_df['Date'] == test_end_date].index[0]

del test_start_date, test_end_date, messages

In [46]:
volatility = {stock: returns_df[stock][test_start: test_end].std() for stock in selected_stocks}


In [47]:
values = np.array(list(volatility.values()))

korelacijska_matrica = returns_df.iloc[:, 1:].corr()

kovarijantska_matrica = korelacijska_matrica * np.outer(values, values)

In [49]:
kovarijantska_matrica

,MSFT,AAPL,AMZN,NVDA,JPM,JNJ,UNH,HD,PG,BAC,DIS,ADBE,XOM,CMCSA,PFE,CSCO,TMO,PEP,VZ
MSFT,0.000550,0.000278,0.000209,0.000360,0.000303,0.000133,0.000192,0.000236,0.000115,0.000272,0.000278,0.000330,0.000271,0.000215,0.000153,0.000294,0.000215,0.000157,0.000119
AAPL,0.000278,0.000650,0.000193,0.000347,0.000261,0.000094,0.000162,0.000197,0.000087,0.000244,0.000230,0.000286,0.000207,0.000168,0.000109,0.000262,0.000183,0.000097,0.000089
AMZN,0.000209,0.000193,0.000451,0.000236,0.000192,0.000055,0.000103,0.000160,0.000046,0.000162,0.000187,0.000227,0.000120,0.000137,0.000082,0.000197,0.000139,0.000066,0.000065
NVDA,0.000360,0.000347,0.000236,0.001091,0.000309,0.000087,0.000164,0.000239,0.000051,0.000276,0.000304,0.000387,0.000254,0.000228,0.000124,0.000354,0.000250,0.000103,0.000105
JPM,0.000303,0.000261,0.000192,0.000309,0.000813,0.000154,0.000259,0.000337,0.000151,0.000700,0.000386,0.000296,0.000379,0.000296,0.000208,0.000288,0.000250,0.000180,0.000161
JNJ,0.000133,0.000094,0.000055,0.000087,0.000154,0.000263,0.000161,0.000127,0.000131,0.000155,0.000153,0.000107,0.000209,0.000120,0.000168,0.000098,0.000125,0.000154,0.000085
UNH,0.000192,0.000162,0.000103,0.000164,0.000259,0.000161,0.000631,0.000207,0.000142,0.000281,0.000226,0.000184,0.000274,0.000187,0.000185,0.000136,0.000194,0.000173,0.000092
HD,0.000236,0.000197,0.000160,0.000239,0.000337,0.000127,0.000207,0.000535,0.000135,0.000312,0.000287,0.000231,0.000253,0.000213,0.000157,0.000206,0.000195,0.000170,0.000115
PG,0.000115,0.000087,0.000046,0.000051,0.000151,0.000131,0.000142,0.000135,0.000298,0.000157,0.000140,0.000095,0.000174,0.000119,0.000126,0.000092,0.000090,0.000168,0.000084
BAC,0.000272,0.000244,0.000162,0.000276,0.000700,0.000155,0.000281,0.000312,0.000157,0.000966,0.000365,0.000276,0.000382,0.000282,0.000213,0.000256,0.000225,0.000173,0.000143
